# Development of miscellaneous new features for polymerist

## Trying out `anytree`

In [3]:
import polymerist as ps
from polymerist import genutils, polymerist, duration
from polymerist.genutils import bits, typetools, importutils
from polymerist.genutils.decorators.functional import allow_string_paths

import json, math
import pkgutil, importlib

print(importutils.module_hierarchy(json))

# from anytree import RenderTree, Node, AsciiStyle, ContStyle, ContRoundStyle, DoubleStyle

├───decoder
├───encoder
├───scanner
└───tool


In [ ]:
from typing import Optional
from anytree import Node
from pathlib import Path


@allow_string_paths
def file_tree_from_path(path : Path, max_depth : Optional[int]=None, _curr_depth : int=0) -> Node:
    '''Compiled a directory tree from a toplevel path
    If recursive=True, will expand each subdirectory node into its own directory subtree'''
    path_node = Node(path.name, path=path, is_dir=path.is_dir(), is_file=path.is_file())
    if path.is_dir() and ( # recursively add subnodes IFF
            (max_depth is None)             # 1) no depth limit is set, or
            or (_curr_depth < max_depth)    # 2) a limit IS set, but hasn't been reached yet
        ): 
        for subpath in path.iterdir():
            subpath_node = file_tree_from_path(subpath, max_depth=max_depth, _curr_depth=_curr_depth+1)
            subpath_node.parent = path_node

    return path_node

In [ ]:
from types import ModuleType
from typing import Iterable, Optional, TypeAlias, Union

import inspect
import pkgutil
import importlib, importlib.machinery
from importlib.machinery import SourceFileLoader, FileFinder

from pathlib import Path
from anytree import Node


# EXTRACTING INFO FROM A SINGLE IMPORTED MODULE
def flexible_module_pass(module : Union[str, Path, ModuleType]) -> ModuleType:
    '''Flexible interface for supplying a ModuleType object as an argument
    Allows for passing a name (either module name or string path), Path location, or a module proper'''
    if isinstance(module, ModuleType):
        return module
    elif isinstance(module, str):
        pass
    elif isinstance(module, Path):
        pass
    else:
        raise TypeError(f'Cannot interpret object of type "{type(module).__name__}" as a module')

def is_package(module : Union[str, ModuleType]) -> bool:
    '''Check whether a Python module is a package (i.e. contains other importable modules within itself)'''
    # module_spec = getattr(module, '__spec__', None) # this doesn't work when a string is passed in
    module_spec = importlib.util.find_spec(module.__name__)
    if module_spec is None:
        raise ValueError(f'No ModuleSpec found for {module}')

    module_loader = module_spec.loader
    # module_loader = pkgutil.get_loader(module) # NOTE: while more compact than above, this function is slated for deprecation
    if module_loader is None:
        raise ValueError(f'No SourcefileLoader found for {module}')

    return module_loader.is_package(module.__name__)

# TODO : find way to get depth of submodule in toplevel ("number of dots" before standalone name)
def relative_module_name(module : ModuleType, relative_to : Optional[ModuleType]=None, remove_leading_dot : bool=True) -> str:
    '''Gets the name of a module relative to another (presumably toplevel) module
    If the given module is not in the path of the toplevel module, will simply return as module.__name__'''
    rel_mod_name = module.__name__
    if relative_to is not None:
        toplevel_prefix = relative_to.__name__
        if remove_leading_dot:
            toplevel_prefix += '.' # append dot to prefix to remove it later
        rel_mod_name = rel_mod_name.removeprefix(toplevel_prefix)

    return rel_mod_name


# COMPILING MODULE TREES FOR FULL PACKAGES
def module_tree(module : ModuleType, blacklist : Optional[Iterable[str]]=None, relative_to : Optional[ModuleType]=None, max_depth : Optional[int]=None, _curr_depth : int=0) -> Optional[Node]:
    '''Create a tree for a module and all its submodules, to a set depth and with optional blacklisting by module name'''
    if blacklist is None:
        blacklist = []

    # TODO: figure out way to get loader (or FileFinder?) for toplevel
    module_is_pkg = is_package(module)
    module_name = relative_module_name(module, relative_to=relative_to, remove_leading_dot=True)

    module_node = Node(module_name, module=module, module_is_pkg=module_is_pkg)
    if module_is_pkg and ( # recursively add subnodes IFF
            (max_depth is None)             # 1) no depth limit is set, or
            or (_curr_depth < max_depth)    # 2) a limit IS set, but hasn't been reached yet
        ):
        for (submodule_loader, submodule_name, sub_is_pkg) in pkgutil.iter_modules(module.__path__):
            if submodule_name not in blacklist: # TOSELF: also worth checking the full __name__? (requires importing a potentially blacklisted module which isn't great)
                submodule = importlib.import_module(f'.{submodule_name}', package=module.__package__)
                submodule_node = module_tree(submodule, blacklist=blacklist, relative_to=module, max_depth=max_depth, _curr_depth=_curr_depth+1)
                submodule_node.parent = module_node

    return module_node


In [ ]:
import json, math
mods = (
    ps,
    polymerist, 
    genutils,
    duration,
    bits,
    typetools,
    json,
    math,
)

for mod in mods:
    print(mod.__name__, is_package(mod))
    print('\t', getattr(mod, '__file__', None), getattr(mod, '__path__', None))
    mspec = mod.__spec__
    print('\t', mspec.origin, mspec.parent, mspec.name)

In [ ]:
from anytree import RenderTree, Node, AsciiStyle, ContStyle, ContRoundStyle, DoubleStyle


mt = module_tree(genutils, blacklist=('decorators',), max_depth=None, relative_to=ps)
rt = RenderTree(mt)
print(rt.by_attr('name'))

In [ ]:
main_path = Path('cleaned_pdbs')
filetree = file_tree_from_path(main_path, max_depth=None)

rt = RenderTree(filetree, style=ContRoundStyle())
print(rt.by_attr('name'))

## Testing polymerist importability within environment

In [ ]:
import numpy as np
from openff.toolkit import Molecule, Topology, ForceField

In [ ]:
import polymerist as ps
from polymerist.genutils import pyimports, importutils

import pandas as pd
print(importutils.module_hierarchy(ps))

In [ ]:
import nglview

print(nglview.__version__)
nglview.demo()

In [ ]:
from polymerist.polymers.monomers import specification
from rdkit import Chem

smi = 'CCO-c1ccccc1-N=C=C'
mol1 = Chem.MolFromSmiles(smi)
display(mol1)

sma = specification.expanded_SMILES(smi, assign_map_nums=True)
exp_sma = specification.compliant_mol_SMARTS(sma)
mol2 = Chem.MolFromSmarts(sma)
display(mol2)


In [ ]:
from openff.toolkit import Molecule

offmol = Molecule.from_smiles(smi)
offmol.generate_conformers(n_conformers=1)
offmol.visualize(backend='nglview')

## Dynamically reading all import statements in codebase

In [ ]:
import polymerist as ps
from polymerist.genutils import pyimports, importutils

print(importutils.module_hierarchy(ps))

In [ ]:
infos = pyimports.extract_imports_from_module(ps)

df = pd.DataFrame.from_records([info.__dict__ for info in infos])
df.to_csv('test.csv')

In [ ]:
nonrel = [info for info in infos if not info.is_relative and info.parent_module is None]
len(nonrel)

In [ ]:
import sys

imported_names = set(info.object_name for info in nonrel)
imported_names

registered_builtins = set(sys.builtin_module_names)
registered_stdlibs = set(sys.stdlib_module_names)

nb_imports = imported_names - registered_builtins - registered_stdlibs
nb_imports

# Another thing

In [1]:
import nglview

nglview.demo()

NGLWidget()